In [15]:
! pip install PyPDF2

Defaulting to user installation because normal site-packages is not writeable


In [7]:
! pip install chardet

Defaulting to user installation because normal site-packages is not writeable


In [180]:
import re
import csv
from datetime import datetime
import PyPDF2
import chardet
import pandas as pd
from enum import Enum
import os

In [171]:
column_types = {
    'date': 'datetime64[ns]',
    'competition_name': 'string',
    'pool_length': 'int64',
    'distance': 'int64',
    'style': 'string',
    'age_category': 'string',
    'athlete': 'string',
    'sex': 'string',
    'age': 'int64',
    'club': 'string',
    'time': 'float64',
    'points': 'int64',
    'place': 'int64'
}

results_dataframe = pd.DataFrame(columns=column_types.keys()).astype(column_types)
print(results_dataframe.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 0 entries
Data columns (total 13 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   date              0 non-null      datetime64[ns]
 1   competition_name  0 non-null      string        
 2   pool_length       0 non-null      int64         
 3   distance          0 non-null      int64         
 4   style             0 non-null      string        
 5   age_category      0 non-null      string        
 6   athlete           0 non-null      string        
 7   sex               0 non-null      string        
 8   age               0 non-null      int64         
 9   club              0 non-null      string        
 10  time              0 non-null      float64       
 11  points            0 non-null      int64         
 12  place             0 non-null      int64         
dtypes: datetime64[ns](1), float64(1), int64(5), string(6)
memory usage: 132.0 bytes
None


In [160]:
class ReadingStatus(Enum):
    READING_STARTED = 1
    DISTANCE_READ = 2
    DATE_READ = 3
    RESULT_READ = 4

class ReadingProgress:
    def __init__(self):
        self.current_status = ReadingStatus.READING_STARTED
        self.status_order = [
            ReadingStatus.READING_STARTED,
            ReadingStatus.DISTANCE_READ,
            ReadingStatus.DATE_READ,
            ReadingStatus.RESULT_READ
        ]
        self.result_read_reached = False

    def set_status(self, new_status: ReadingStatus):
        if self.result_read_reached: # Если RESULT_READ был установлен, то разрешаем любые переходы
            self.current_status = new_status
            return

        """Устанавливает новый статус только в том случае, если он следующий в последовательности."""
        try:
            current_index = self.status_order.index(self.current_status)
            next_index = current_index + 1
            if next_index < len(self.status_order) and self.status_order[next_index] == new_status:
                self.current_status = new_status
                if new_status == ReadingStatus.RESULT_READ:  # Если достигли RESULT_READ, то устанавливаем флаг
                   self.result_read_reached = True
        except ValueError as e:
            pass #Просто ничего не делаем, если статус нельзя установить

    def is_result_read(self) -> bool:
        """Возвращает True, если текущий статус RESULT_READ."""
        return self.current_status == ReadingStatus.RESULT_READ

In [167]:
    date = pd.to_datetime('01-01-1993')
    start_result = False
    start_distance = False

    full_text = ""
    try:
      with open(file_path, 'rb') as pdf_file:
        pdf_reader = PyPDF2.PdfReader(pdf_file)
        for page_num in range(len(pdf_reader.pages)):
            page = pdf_reader.pages[page_num]
            full_text += page.extract_text()
    except Exception as e:
       print(f"Ошибка при открытии или чтении PDF-файла: {e}")
    
    date_pattern = r"(\d{2}.\d{2}.\d{4})"
    result_pattern = r"(\d+)\.(\w+\s+\w+)\s+(\d+)\s+(\w+)\s*(\w+)?\s+([\d:\.]+)\s+(\w+)"
    race_pattern = r"(Женщины|Мужчины)\s+,\s+(\w+)m\s+(\w+)\s+(\w+\s)?(\d+\s-\s\d+)"

    # Определяем кодировку текста
    raw_text = full_text.encode('utf-8', errors='ignore')
    result = chardet.detect(raw_text)
    encoding = result['encoding']

    if encoding:
       full_text = raw_text.decode(encoding, errors='replace')
    else:
        full_text = raw_text.decode('utf-8', errors='replace')
    
    for line in full_text.splitlines():
        line = line.strip()
        if "ГРИГОРЬЕВА" in line:
            print(line)

4.ГРИГОРЬЕВА Елена 63 Драфт 1:44.42 526 48.63 55.79
3.ГРИГОРЬЕВА Анна 30 Свим -Изи СПб 1:31.11 443 43.39 47.72
ПРАВДИНА Елена 40 +0,40 36.14 ГРИГОРЬЕВА Анна 30 +0,36
6.ГРИГОРЬЕВА Анна 30 Свим -Изи СПб 32.54 489
6.ГРИГОРЬЕВА Елена 63 Драфт 46.84 536
4.ГРИГОРЬЕВА Анна 30 Свим -Изи СПб 1:23.33 436 38.39 44.94
ГРИГОРЬЕВА Анна 30 41.96 ДОРОНИНА Марина 42 31.69


In [175]:
def parse_pdf_swimming_results(file_path, competition_name, pool_length):
    """
    Извлекает данные из PDF файла и создает CSV файл с результатами соревнований.

    Args:
    file_path (str): Путь к PDF файлу с результатами.
    """

    full_text = ""
    try:
      with open(file_path, 'rb') as pdf_file:
        pdf_reader = PyPDF2.PdfReader(pdf_file)
        for page_num in range(len(pdf_reader.pages)):
            page = pdf_reader.pages[page_num]
            full_text += page.extract_text()
    except Exception as e:
       print(f"Ошибка при открытии или чтении PDF-файла: {e}")
       return
    
    date_pattern = r"(\d{2}.\d{2}.\d{4})"
    result_pattern = r"(\d+)\.(\w+\s+\w+)\s+(\d+)\s+(\w+)\s*(\w+)?\s+([\d:\.]+)\s+([0-9]{3,4})"
    race_pattern = r"(Женщины|Мужчины)\s+,\s+(\w+)m\s+(\w+)\s+(\w+\s)?(\d+\s-\s\d+)"

    progress = ReadingProgress()

    # Определяем кодировку текста
    raw_text = full_text.encode('utf-8', errors='ignore')
    result = chardet.detect(raw_text)
    encoding = result['encoding']

    if encoding:
       full_text = raw_text.decode(encoding, errors='replace')
    else:
        full_text = raw_text.decode('utf-8', errors='replace')
    
    for line in full_text.splitlines():
        line = line.strip()
        if re.search(r"^\d{1,2}\.[а-яА-Я]{2,}", line):
          match = re.search(result_pattern, line)
          if match:
            progress.set_status(ReadingStatus.RESULT_READ)
            place = int(match.group(1))
            athlete = match.group(2)
            age = int(match.group(3))
            club = match.group(4)
            if match.group(5):
              club += ' ' + match.group(5)
            time = match.group(6)
            points = match.group(7)

        elif "Дистанция" in line:
          match = re.search(race_pattern, line)
          if match:
            progress.set_status(ReadingStatus.DISTANCE_READ)
            if match.group(1) == 'Женщины':
              sex = 'ж'
            else:
              sex = 'м'
            distance = int(match.group(2))
            style = match.group(3)
            if match.group(4):
              style += ' ' + match.group(4)  
            age_category = match.group(5)  

        elif "Результаты" in line:
          # re.search(r"202\d", line) and 
            match = re.search(date_pattern, line)
            if match:
              progress.set_status(ReadingStatus.DATE_READ)
              date = match.group(1)

        if progress.is_result_read():
          new_row_data = {
            'date':  date,
            'competition_name': competition_name,
            'pool_length': pool_length,
            'distance': distance,
            'style': style,
            'age_category': age_category,
            'athlete': athlete,
            'sex': sex,
            'age': age,
            'club': club,
            'time': time,
            'points': points,
            'place': place
          }

          results_dataframe.loc[len(results_dataframe)+1] = new_row_data
        # print(start)

        # if start:
        #   print(datetime.strptime(date, "%d.%m.%Y").date())
        
          
    

In [147]:
competition_name = "Рождественский Кубок 2025"
pool_length = "25"

In [176]:
type(results_dataframe)

pandas.core.frame.DataFrame

In [170]:
results_dataframe = results_dataframe.drop(results_dataframe.index)
results_dataframe = results_dataframe.reset_index(drop=True, inplace=True)

In [178]:
file_path = 'sources/Рождественский_Кубок_2025.pdf'
parse_pdf_swimming_results(file_path, competition_name, pool_length)
results_dataframe = results_dataframe.drop_duplicates().reset_index(drop=True)

In [181]:
file_path = 'sources/results_dataframe.csv'

if os.path.exists(file_path) and os.path.getsize(file_path) > 0: #проверка что файл не пустой
    header = False
    mode = 'a'
else:
    header = True
    mode = 'w'

results_dataframe.to_csv(
    path_or_buf=file_path,
    sep=',',
    na_rep='',
    header=header,
    index=True,
    mode=mode,
    encoding='windows-1251',
    compression='infer',
    quotechar='"',
    doublequote=True,
    decimal='.',
    errors='strict',
)

In [185]:
! pip install --upgrade pandas

Defaulting to user installation because normal site-packages is not writeable

In [184]:
test_line = "3.ГРИГОРЬЕВА Анна 30 Голд Свим Мастерс 1:31.11 443 43.39 47.72"
pattern = r"(\d+)\.(\w+\s+\w+)\s+(\d+)\s+(\w+)\s*([\w\s-]+)\s+([\d:\.]+)\s+([0-9]{3,4})"
match = re.search(pattern, test_line)
if match:
    place = match.group(1)
    name = match.group(2)
    age = match.group(3)
    club = match.group(4)
    if match.group(5):
        club += ' ' + match.group(5)
    time = match.group(6)
    points = match.group(7)
else:
    print("no")
print(club)

Голд Свим Мастерс


In [101]:
test_line = "Дистанция 19 Мужчины , 100m Вольный стиль 50 - 54 лет"
pattern = r"(Женщины|Мужчины)\s+,\s+(\w+)m\s+(\w+)\s+(\w+\s)?(\d+\s-\s\d+)"
match = re.search(pattern, test_line)
if match:
    if match.group(1) == 'Женщины':
        sex = 'ж'
    else:
        sex = 'м'
    distance = match.group(2)
    style = match.group(3)
    if match.group(4):
        style += ' ' + match.group(4).rstrip()  
    age_category = match.group(5)  
else:
    print("no")
print(style + '!')

Вольный стиль!


In [31]:
test_line = "05.01.2025 - 13:00 Результаты"
pattern = r"(\d{2}.\d{2}.\d{4})"
match = re.search(pattern, test_line)
if match:
    date = match.group(1)
else:
    print("no")
print(datetime.strptime(date, "%d.%m.%Y").date())

2025-01-05


In [169]:
competition_name = "Рождественский Кубок 2025"
pool_length = "25"

In [52]:
test_line = "1.ПЕТКАУ Олег 78 Невские звёзды 1:19.57 666 39.26 40.31"
if re.search(r"^\d{1,2}\.[а-яА-Я]{2,}", test_line):
    print(1)
else:
    print(0)

1
